In [1]:
import Bio
from Bio import Phylo
from Bio import AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
import pylab
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pydot
import pygraphviz as pgv
from networkx.drawing.nx_pydot import graphviz_layout
from networkx import all_pairs_shortest_path_length
import hyperlib
from hyperlib.embedding.treerep import treerep
from hyperlib.embedding.sarkar import sarkar_embedding
from hyperlib.utils.multiprecision import poincare_dist
from hyperlib.utils.multiprecision import poincare_reflect0
#from hyperlib.manifold.lorentz import Lorentz
#from hyperlib.manifold.poincare import Poincare
import mpmath as mpm
from scipy.special import expit
import tensorflow as tf
from tensorflow import keras

In [2]:
#pip install hyperlib --user

In [3]:
#pip install tensorflow-datasets

In [4]:
#conda install --channel conda-forge pygraphviz

In [5]:
#conda update -n base -c defaults conda

In [6]:
file_name = './205_na_aln.fa'
file_format = 'fasta'

In [7]:
aln = AlignIO.read(file_name, file_format)
#print(aln)

In [8]:
labels = {i:seq.id for i, seq in enumerate(aln)}
#labels

In [9]:
aln_lenght = aln.get_alignment_length()

sotto il modulo treeconstruction di Phylo c'è questo DistanceCalculator che calcola automaticamente le distanze secondo un modello dato come input, 'identity' di default.
il metodo get_distance produce un oggetto di tipo DistanceMatrix, che va quindi converito con numpy.

In [10]:
calc = DistanceCalculator('trans')
dm = calc.get_distance(aln)
#print(dm)
type(dm)

Bio.Phylo.TreeConstruction.DistanceMatrix

In [11]:
dist_mat = np.array(dm)
#dist_mat

In [12]:
#tree = Phylo.read('strain_tree.nwk', 'newick')
#tree.ladderize()
#Phylo.draw(tree)
#type(tree)
#leaves = tree.get_terminals()
#dist_mat = np.array([[tree.distance(leaf1, leaf2 ) for leaf2 in leaves] for leaf1 in leaves])
#dist_mat

In [13]:
constructor = DistanceTreeConstructor()
ptree = constructor.upgma(dm)
ptree.ladderize()
#Phylo.draw(ptree)

In [14]:
#Phylo.draw(ptree)

In [15]:
htree = treerep(dist_mat,return_networkx = True, tol = 0.0000000000000000000000001)
htree


In [16]:
htree.number_of_nodes()

224

In [17]:
pesi_aggiornati = [(edge[0], edge[1], {'weight' : expit(htree.get_edge_data(edge[0], edge[1])['weight'])}) for edge in list(htree.edges)]
#pesi_aggiornati

In [18]:
for i in pesi_aggiornati:
    htree.update([i])
htree.get_edge_data(4, 49)

In [19]:
htree[3]

AtlasView({33: {'weight': 0.5}})

In [20]:
htgraph = nx.nx_agraph.to_agraph(htree)

In [21]:
#htgraph.layout?

In [22]:
htgtwopi = htgraph
htgtwopi.layout(prog = 'twopi') #dot, neato
#htgtwopi

In [23]:
print(htree)

Graph with 224 nodes and 223 edges


Infine il codice suggerito nell'esempio della libreria su github che dovrebbe fare l'embedding iperbolico nella sfera di Poincaré

In [24]:
root = 0 # label of root node
tau = 0.2 # scaling factor for edges
embed_2D = sarkar_embedding(htree, root)

# calculate hyperbolic distances from the embedding
poincare_dist(embed_2D[0,:], embed_2D[1,:])

mpf('182.3930140184730296830577818620853629355128624706745952553613730108328677888921117463192282011772899928295324397826630587037104171664147009266325151865162568555755224573700242316251616213378906730657917555326376907130667781711161946561635505501557502749903012449283476284226224131427363125102892699180013462569534431948250584')

In [25]:
#embed_2D

In [26]:
embed_2D[2, :] - embed_2D[1, :]

matrix(
[['-0.000003423767373985497984751829356124747085063760477591075834961990692450826259296341224306019947944315079684713212979073356493723219441847436171787303741494917810710510838979120572783480491943591745672352197053777428079487765954959897686032287107621031035249424852628182889140779419344143473784514951479874114276130901639076456', '0.005231774456298183848868269141325044776261945825202421062376019137696801914050044753010894185013976064400681042387321046914197558385284662849526699172849311596174751965364629275793968615729134769901644605393966053989845375783867845587629778476655030500633610682487087931944236830809385226817972148091128030929594756482837622']])

# **Codice di preparazione per H-depp**

In [27]:
import sys 
from pathlib import Path

module_path = str(Path.cwd().parents[0])
sys.path.append(module_path)

In [28]:
from src.data_generator import DataGenerator

In [29]:
tree_dist = dict(all_pairs_shortest_path_length(htree))

In [30]:
tree_dist[0][3]

2

In [31]:
emb_dist = {}

#note that i use poincare_dist to calculate the embeddings distance
#we might want to calculate it differently
for i in range (0, embed_2D.rows):
    temp_dict = {}
    for j in range (0, embed_2D.rows):
        jnode = list(htree.nodes)[j]
        temp_dict[jnode] = poincare_dist(embed_2D[i,:], embed_2D[j,:])
    inode = list(htree.nodes)[i]
    emb_dist[inode] = temp_dict

emb_dist[0][3]

mpf('183.5706887125437275012669761316737293472617396839872234836643717924866364940717333990487804252432743470169131907996470112513843975258455353536347153842456837294554564308593777892772591629778012512753187676099243081539128496201698145316497961309813613126520098908361853240154806379715122640915085424493210542363988208267386476')

In [32]:
def distortion_func(e_dist, t_dist):
    if (t_dist != 0):
        return np.square(( e_dist / t_dist ) - 1) 
    else: return 0

In [33]:
distortion = {}

for i in range (0, embed_2D.rows):
    temp_dict = {}
    for j in range (0, embed_2D.rows):
        jnode = list(htree.nodes)[j]
        temp_dict[jnode] = distortion_func(emb_dist[i][j], tree_dist[i][j])
    inode = list(htree.nodes)[i]
    distortion[inode] = temp_dict
    
distortion[0][3]

mpf('4.95021677894868878817336334564908546740134272664065431651070898600347645811634249429357125024968430692939913138850955383342030083736693737043879145601905638699703476019557666213811223948491572384014138730421419884895025299168795358490885744883002682359412122892913711795255528735768967170928689282513655583160863458363731869')

In [34]:
aln[0].seq

Seq('ATGAGCCAAGAAGAAAAGTTACCAAAGATTCTGATCGTTGAAGACGACGAGCGT...TTG')

In [35]:
hot_dict = {'A' : np.array([0, 0, 0, 1]),
           'T' : np.array([0, 0, 1, 0]),
           'G' : np.array([0, 1, 0, 0]),
           'C' : np.array([1, 0, 0, 0]),
           '-' : np.array([0, 0, 0, 0]),}
hot_dict[aln[0].seq[2]]

array([0, 1, 0, 0])

In [36]:
encoded_seqs = [np.array([hot_dict[car] for car in alignment.seq]) for alignment in aln]

In [37]:
encoded_seqs[28]

array([[0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       ...,
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

In [38]:
original_nodes = len(aln)

In [39]:
model_inputs = [[(encoded_seqs[i], encoded_seqs[j], tree_dist[i][j]) for j in tree_dist[i] if j < original_nodes] for i in tree_dist if i < original_nodes]

In [40]:
model_inputs[0][3][0]

array([[0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       ...,
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

In [41]:
params = {'dim': (aln_lenght, ),
          'batch_size': 128,
          'n_classes': 714, #shouldn't be categorical
          'n_channels': 4,
          'shuffle': True}

In [42]:
#lista_IDs_train = [[i, j] for j in range(0, 22) for i in range(0, 22)]
#lista_IDs_val = [[i, j] for j in range(22, 29) for i in range(22, 29)] #what should I use for validation?

In [43]:
lista_IDs_train = [i for i in range(0, original_nodes-30)]

In [44]:
lista_IDs_val = [i for i in range(original_nodes-30, original_nodes)]

In [45]:
train_ds_gen = DataGenerator(lista_IDs_train, model_inputs, **params)
test_ds_gen = DataGenerator(lista_IDs_val, model_inputs, **params)

In [46]:
iteratore = iter(train_ds_gen)
#next(iteratore)

In [47]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(714, 4)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Embedding(714, 2)
    #tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    #sparse vs non sparse
)

model.fit(
    train_ds_gen,
    epochs=9,
    validation_data = test_ds_gen
)

Epoch 1/9


ValueError: in user code:

    File "C:\Users\aster\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\aster\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\aster\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\aster\anaconda3\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\aster\anaconda3\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\aster\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "